# Tracing CrewAI Pipelines

This notebook shows you how to add **complete observability** to CrewAI multi-agent pipelines—tracing agent-to-agent handoffs, measuring individual agent performance, and tracking per-agent costs.

**What You'll Learn:**
- Capture the message flow between agents as tasks pass through the pipeline
- Monitor token usage and costs for each agent role to identify cost drivers
- Understand why agents made specific decisions and where quality degrades
- Run experiments with different model assignments to find the cost/quality sweet spot

**Note:** All company names (ContentCraft) and scenarios in this cookbook are entirely fictional and used for demonstration purposes only.

**Prerequisites:**
- Python 3.9+
- OpenAI API key
- Netra API key ([Get started here](https://docs.getnetra.ai/quick-start/Overview))
- CrewAI installed

## Step 0: Install Packages

In [ ]:
pip install netra-sdk crewai crewai-tools openai langchain-openai

## Step 1: Set Environment Variables

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key:")
os.environ["NETRA_API_KEY"] = getpass("Enter your Netra API Key:")
os.environ["NETRA_OTLP_ENDPOINT"] = getpass("Enter your Netra OTLP Endpoint:")

print("API keys configured!")


## Step 2: Initialize Netra for Multi-Agent Tracing

In [ ]:
from netra import Netra
from netra.instrumentation.instruments import InstrumentSet

# Initialize Netra with CrewAI and OpenAI instrumentation
Netra.init(
    app_name="content-crew",
    headers=f"x-api-key={os.getenv('NETRA_API_KEY')}",
    environment="development",
    trace_content=True,
    instruments={InstrumentSet.OPENAI},  # CrewAI uses OpenAI under the hood
)

print("Netra initialized for multi-agent tracing!")

## Step 3: Define the CrewAI Agents

Create a 4-agent content pipeline: Researcher → Writer → Editor → SEO

In [ ]:
from crewai import Agent
from langchain_openai import ChatOpenAI

def create_agents(config: dict = None):
    """Create the content team agents with configurable models."""
    config = config or {
        "researcher": "gpt-4o-mini",
        "writer": "gpt-4o-mini",
        "editor": "gpt-4o-mini",
        "seo": "gpt-4o-mini",
    }

    researcher = Agent(
        role="Research Specialist",
        goal="Gather accurate facts, statistics, and expert opinions",
        backstory="Expert researcher with 10 years of experience in content research.",
        llm=ChatOpenAI(model=config["researcher"], api_key=os.getenv("OPENAI_API_KEY")),
        verbose=True,
    )

    writer = Agent(
        role="Content Writer",
        goal="Write engaging, well-structured blog articles",
        backstory="Professional copywriter with expertise in compelling content.",
        llm=ChatOpenAI(model=config["writer"], api_key=os.getenv("OPENAI_API_KEY")),
        verbose=True,
    )

    editor = Agent(
        role="Quality Editor",
        goal="Polish articles for clarity, grammar, and flow",
        backstory="Senior editor with a keen eye for detail.",
        llm=ChatOpenAI(model=config["editor"], api_key=os.getenv("OPENAI_API_KEY")),
        verbose=True,
    )

    seo_specialist = Agent(
        role="SEO Optimizer",
        goal="Optimize content for search engines",
        backstory="SEO expert who balances keywords with readability.",
        llm=ChatOpenAI(model=config["seo"], api_key=os.getenv("OPENAI_API_KEY")),
        verbose=True,
    )

    return {
        "researcher": researcher,
        "writer": writer,
        "editor": editor,
        "seo": seo_specialist,
    }


print("Agent creation function defined!")

## Step 4: Define the Tasks

Create tasks that chain together through the agents.

In [ ]:
from crewai import Task

def create_tasks(agents: dict, topic: str):
    """Create the content pipeline tasks."""

    research_task = Task(
        description=f"Research the topic: '{topic}'. Find key facts and statistics.",
        expected_output="Research brief with facts, statistics, and sources",
        agent=agents["researcher"],
    )

    writing_task = Task(
        description="Write a 500-800 word blog article based on the research.",
        expected_output="Draft blog article in markdown format",
        agent=agents["writer"],
        context=[research_task],
    )

    editing_task = Task(
        description="Edit the article for grammar, flow, and clarity.",
        expected_output="Polished blog article with improved clarity",
        agent=agents["editor"],
        context=[writing_task],
    )

    seo_task = Task(
        description="Optimize the article for SEO with meta description and keywords.",
        expected_output="SEO-optimized article with metadata",
        agent=agents["seo"],
        context=[editing_task],
    )

    return [research_task, writing_task, editing_task, seo_task]


print("Task creation function defined!")

## Step 5: Create the Crew

Assemble the agents and tasks into a crew that executes the pipeline.

In [ ]:
from crewai import Crew, Process
from netra.decorators import workflow

def run_content_crew(topic: str, config: dict = None):
    """Execute the content creation pipeline."""
    agents = create_agents(config)
    tasks = create_tasks(agents, topic)

    crew = Crew(
        agents=list(agents.values()),
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )

    print(f"\n{'='*60}")
    print(f"Running Content Creation Pipeline")
    print(f"Topic: {topic}")
    print(f"{'='*60}\n")

    result = crew.kickoff()

    return result


print("Crew execution function defined!")

## Step 6: Run the Pipeline

Execute the content pipeline with auto-tracing enabled.

In [ ]:
# Run the default configuration (all agents using gpt-4o-mini)
topic = "The Future of Artificial Intelligence in Healthcare"
result = run_content_crew(topic)

print("\n" + "="*60)
print("Pipeline Complete!")
print("="*60)
print("\nFinal Output:")
print(result)

## Step 7: Compare Configuration Experiments

Test different model assignments to optimize cost vs. quality.

In [ ]:
# Configuration 1: Premium (all GPT-4o-mini)
print("\n" + "="*60)
print("Configuration 1: Premium")
print("="*60)

premium_config = {
    "researcher": "gpt-4o-mini",
    "writer": "gpt-4o-mini",
    "editor": "gpt-4o-mini",
    "seo": "gpt-4o-mini",
}

premium_result = run_content_crew(
    "The Impact of Renewable Energy on Climate Change",
    config=premium_config
)

In [ ]:
# Configuration 2: Budget (lighter models)
print("\n" + "="*60)
print("Configuration 2: Budget")
print("="*60)

budget_config = {
    "researcher": "gpt-4o-mini",
    "writer": "gpt-4o-mini",
    "editor": "gpt-4o-mini",
    "seo": "gpt-4o-mini",
}

budget_result = run_content_crew(
    "The Role of Machine Learning in Modern Business",
    config=budget_config
)

## Step 8: Track Agent Performance

Record per-agent metrics and costs in your traces.

In [ ]:
import time
from netra.decorators import task

class TrackedAgentSession:
    """Track multi-agent pipeline execution metrics."""

    def __init__(self, pipeline_name: str):
        self.pipeline_name = pipeline_name
        self.metrics = {
            "researcher": {"start": None, "end": None, "duration": 0},
            "writer": {"start": None, "end": None, "duration": 0},
            "editor": {"start": None, "end": None, "duration": 0},
            "seo": {"start": None, "end": None, "duration": 0},
        }

    def record_agent_start(self, agent_name: str):
        """Record when an agent starts execution."""
        if agent_name in self.metrics:
            self.metrics[agent_name]["start"] = time.time()

    def record_agent_end(self, agent_name: str):
        """Record when an agent completes."""
        if agent_name in self.metrics:
            self.metrics[agent_name]["end"] = time.time()
            start = self.metrics[agent_name]["start"]
            end = self.metrics[agent_name]["end"]
            if start and end:
                self.metrics[agent_name]["duration"] = end - start

    def print_summary(self):
        """Print performance summary."""
        print(f"\nAgent Performance Summary:")
        print("-" * 40)
        for agent, data in self.metrics.items():
            duration = data["duration"]
            print(f"{agent:15} {duration:8.2f}s")
        total = sum(d["duration"] for d in self.metrics.values())
        print("-" * 40)
        print(f"{'Total':15} {total:8.2f}s")


# Example usage
session = TrackedAgentSession("content-pipeline")
print("Agent performance tracking session created!")

---

## What You'll See in the Dashboard

After running this notebook, check the Netra dashboard for:

- **Per-agent spans** showing execution time for each role
- **Agent handoffs** showing context flow between agents
- **Token usage breakdown** by agent and model
- **Cost attribution** to each agent in the pipeline
- **Configuration comparison** metrics across different model assignments

## Key Metrics for Multi-Agent Systems

| Metric | What It Reveals | How to Use It |
|--------|-----------------|---------------|
| Per-agent latency | Which agent is slow | Optimize bottleneck agent |
| Token usage by agent | Which agent costs most | Downgrade expensive agents |
| Task output quality | Where quality degrades | Adjust agent instructions |
| Handoff losses | Context loss between agents | Improve prompt templates |

## Documentation Links

- [Netra Documentation](https://docs.getnetra.ai)
- [Multi-Agent Observability](https://docs.getnetra.ai/Observability/Agents)
- [CrewAI Integration](https://docs.getnetra.ai/Integrations/orchestrators/CrewAI)
- [Traces Overview](https://docs.getnetra.ai/Observability/Traces/overview)

## See Also

- [Tracing LangChain Agents](/Cookbooks/observability/tracing-langchain-agents) - Single-agent ReAct tracing
- [Tracing RAG Pipeline](/Cookbooks/observability/tracing-rag-pipeline) - Apply patterns to RAG systems
- [Multi-Tenant Cost Tracking](/Cookbooks/observability/multi-tenant-cost-tracking) - Track per-customer costs